In [11]:
import pandas as pd
import numpy as np
import pyarrow as pa
import re
import findspark
findspark.init('/home/dave/spark-2.4.1-bin-hadoop2.7/')
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import *
import sys
import os
from IPython.display import display, HTML
from pyspark.sql.functions import pandas_udf, PandasUDFType

datapath = "/home/dave/Documents/CS691/proj/data/snotel/"
s1049="SNOTEL_1049_FORESTDALE_CREEK"    
s539="SNOTEL_539_INDEPENDENCE_CAMP"
s1050="SNOTEL_1050_HORSE_MEADOW"        
s540="SNOTEL_540_INDEPENDENCE_CREEK"
s1051="SNOTEL_1051_BURNSIDE_LAKE"       
s541="SNOTEL_541_INDEPENDENCE_LAKE"
s1067="SNOTEL_1067_CARSON_PASS"         
s615="SNOTEL_615_MARLETTE_LAKE"
s340="SNOTEL_340_BIG_MEADOW"           
s652="SNOTEL_652_MT_ROSE"
s356="SNOTEL_356_BLUE_LAKES"          
s428="SNOTEL_428_CSS_LAB"              
s724="SNOTEL_724_RUBICON"
s809="SNOTEL_809_TAHOE_CITY"
s463="SNOTEL_463_ECHO_PEAK"            
s834="SNOTEL_834_TRUCKEE"
s508="SNOTEL_508_HAGANS_MEADOW"        
s848="SNOTEL_848_WARD_CREEK"
s518="SNOTEL_518_HEAVENLY_VALLEY"



truckee = [s541,s540,s428]
west_shore = [s809,s848,s724]
south_shore = [s518,s508,s1050,s463]
mt_rose = [s340,s652,s615]
hope_valley = [s1051,s1049,s1067,s356]


In [49]:
df541 = pd.read_csv(s541+".csv")

df540 = pd.read_csv(s540+".csv")

df428 = pd.read_csv(s428+".csv")

df541 = df541.merge(df540, on = "date_only")
truckee = df541.merge(df428, on = "date_only")
truckee =truckee.loc[:, ~truckee.columns.str.contains('^Unnamed')]
truckee['mean'] = truckee.mean(axis =1)
truckee['mean'] = truckee['mean'].round()


truckee = truckee.drop(['max(total_av_corrected)_x','max(total_av_corrected)_y','max(total_av_corrected)'], axis =1)

truckee.info()
truckee


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,2.0
1,2009-04-02,3.0
2,2009-04-03,3.0
3,2009-04-04,4.0
4,2009-04-05,4.0
5,2009-04-06,4.0
6,2009-04-07,4.0
7,2009-04-08,2.0
8,2009-04-09,2.0
9,2009-04-10,2.0


In [19]:
truckee


,Unnamed: 0_x,date_only,max(total_av_corrected)_x,Unnamed: 0_y,max(total_av_corrected)_y,Unnamed: 0,max(total_av_corrected),mean
0,0,2009-04-01,2.0,0,2.0,0,3.0,1.166667
1,1,2009-04-02,1.0,1,3.0,1,5.0,2.000000
2,2,2009-04-03,2.0,2,1.0,2,5.0,2.333333
3,3,2009-04-04,2.0,3,5.0,3,5.0,3.500000
4,4,2009-04-05,2.0,4,5.0,4,4.0,3.833333
5,5,2009-04-06,2.0,5,5.0,5,5.0,4.500000
6,6,2009-04-07,2.0,6,5.0,6,5.0,5.000000
7,7,2009-04-08,1.0,7,2.0,7,4.0,4.666667
8,8,2009-04-09,1.0,8,1.0,8,5.0,5.166667
9,9,2009-04-10,1.0,9,1.0,9,5.0,5.666667


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,2.0
1,1,2009-04-02,1.0
2,2,2009-04-03,2.0
3,3,2009-04-04,2.0
4,4,2009-04-05,2.0
5,5,2009-04-06,2.0
6,6,2009-04-07,2.0
7,7,2009-04-08,1.0
8,8,2009-04-09,1.0
9,9,2009-04-10,1.0


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,2.0
1,1,2009-04-02,3.0
2,2,2009-04-03,1.0
3,3,2009-04-04,5.0
4,4,2009-04-05,5.0
5,5,2009-04-06,5.0
6,6,2009-04-07,5.0
7,7,2009-04-08,2.0
8,8,2009-04-09,1.0
9,9,2009-04-10,1.0


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,3.0
1,1,2009-04-02,5.0
2,2,2009-04-03,5.0
3,3,2009-04-04,5.0
4,4,2009-04-05,4.0
5,5,2009-04-06,5.0
6,6,2009-04-07,5.0
7,7,2009-04-08,4.0
8,8,2009-04-09,5.0
9,9,2009-04-10,5.0


,Unnamed: 0,date_only,max(total_av_corrected)
